## Altmap Experiments
### Compare altmap to map eq using networkx


In [1]:
from matplotlib.ticker import (AutoMinorLocator)
from numpy import log2 as ld

from altmap.altmap_helpers.general import *

# show plots in separate window
%pylab

# init rc params
init_plt_params()

def generate_two_rings(n_ring=10):
    N = 2 * n_ring  # num nodes
    G = nx.Graph()
    G.add_nodes_from(range(1, N + 1))
    
    edges = [(n, n+1) for n in range(1,N)]
    edges += [(1, int(N / 2)), (int(N / 2) + 1, N)]
    G.add_edges_from(edges)
    
    return G

# compute objective for given ring size
def two_rings_cost(ring_size = 3, c1 = 2, c2 = 8):
    n = ring_size
    m = 2*n+1 # number of edges in the network

    J_true = np.log2(m) - 1.0 - (m-1) / m * np.log2(m-1)
    J_init = (2*(n-1)*ld(1-1/m)+3*ld(1-3/(2*m)))/m
    # J_lower_bound = -(3 - 3*np.log2(3) + m*np.log2(m))/m
    alpha = (n%2 != 0) # 1 if odd (3 ind sets), 0 if even (2 ind sets)
    J_ind = -(1 - np.log2(1+2*alpha/m) - 2*alpha/m*np.log2((m-2*alpha)/(m+2*alpha)) - 2*alpha/m)
    
    # c1 modules per ring
    if c1 <= n:
        c=c1
        nm = int(n/c); delta = n-c*nm
        p1=(nm+1)/m; p1not1=1/m; p2=nm/m; p2not2=1/m; p3=(2*nm+1)/(2*m); p3not3=3/(2*m)
        J_c1_comms = 2*(c-delta-1)*altmap_module_cost(p2, p2not2)\
                     + 2*altmap_module_cost(p3, p3not3)
        if delta > 0:
            J_c1_comms += 2*delta*altmap_module_cost(p1, p1not1)
    else:
        J_c1_comms = None
        
    # c2 modules per ring
    if c2 <= n:
        c=c2
        nm = int(n/c); delta = n-c*nm
        p1=(nm+1)/m; p1not1=1/m; p2=nm/m; p2not2=1/m; p3=(2*nm+1)/(2*m); p3not3=3/(2*m)
        J_c2_comms = 2*(c-delta-1)*altmap_module_cost(p2, p2not2)\
                     + 2*altmap_module_cost(p3, p3not3)
        if delta > 0:
            J_c2_comms += 2*delta*altmap_module_cost(p1, p1not1)
    else:
        J_c2_comms = None
    
    return J_init, J_true, J_ind, J_c1_comms, J_c2_comms

# compute altmap objective for given ring size as a function of the number of ring segments
def two_rings_cost_c_modules(ring_size = 3):
    n = ring_size
    m = 2*n+1 # number of edges in the network
    
    c_list = np.asarray(range(2,31))
    J_c_comms_list = []
    for i,c in enumerate(c_list):
        if c <= n:
            nm = int(n/c); delta = n-c*nm
            p1=(nm+1)/m; p1not1=1/m; p2=nm/m; p2not2=1/m; p3=(2*nm+1)/(2*m); p3not3=3/(2*m)
            J_c_comms = 2*(c-delta-1)*altmap_module_cost(p2, p2not2)\
                         + 2*altmap_module_cost(p3, p3not3)
            if delta > 0:
                J_c_comms += 2*delta*altmap_module_cost(p1, p1not1)
            J_c_comms = -J_c_comms
        else:
            J_c_comms = None
        
        J_c_comms_list.append(J_c_comms)
    
    return J_c_comms_list, c_list


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
n_ring = 20
N = 2*n_ring # num nodes
G = generate_two_rings(n_ring)

# compute altmap cost for different community setups
nodes_ids = list(range(1, N+1))
ring_half1_size = int(n_ring/2)
ring_half2_size = n_ring - ring_half1_size

# initial cost
communities = dict(zip(nodes_ids, nodes_ids))
cost = altmap_cost(G, communities)
print (f'Initial Cost L = {cost}\n')

# ground truth
labels = [1] * n_ring + [2] * n_ring
communities = dict(zip(nodes_ids, labels))
cost = altmap_cost(G, communities)
print (f'Ground Truth Cost L = {cost}\n')

# 2 mixed cliques
labels = [1] * ring_half1_size + [2] * ring_half2_size + [1] * ring_half2_size + [2] * ring_half1_size
communities = dict(zip(nodes_ids, labels))

cost = altmap_cost(G, communities)
print (f'Mixed Communities Cost L = {cost}\n')

# 4 mixed cliques
labels = [1] * ring_half1_size + [2] * ring_half2_size + [3] * ring_half1_size + [4] * ring_half2_size
communities = dict(zip(nodes_ids, labels))

cost = altmap_cost(G, communities)
print (f'Four Mixed Communities Cost L = {cost}\n')

Initial Cost L = -0.03672888809801006

Ground Truth Cost L = -0.3633091183255949

Mixed Communities Cost L = -0.20072084242720778

Four Mixed Communities Cost L = -0.8176091626494082



In [ ]:
modules1_per_ring=2
modules2_per_ring=8

n_list = np.logspace(0,3,30, endpoint=False, dtype=int)
n_list = list(map(lambda x: x+x%2, n_list))
n_list = np.unique(n_list) + 3

J_init_list = np.zeros((len(n_list), 1))
J_true_list = np.zeros_like(J_init_list)
J_ind_even_list = np.zeros_like(J_init_list)
J_ind_odd_list = np.zeros_like(J_init_list)
J_c1_comms_list = np.zeros_like(J_init_list)
J_c2_comms_list = np.zeros_like(J_init_list)
J_altmap_list = np.zeros_like(J_init_list)
J_altmap_sci_list = np.zeros_like(J_init_list)
for i, n in enumerate(n_list):
    print (f'Iteration {i+1}/{len(n_list)}')
    J_init_list[i], J_true_list[i], J_ind_odd_list[i], J_c1_comms_list[i],\
    J_c2_comms_list[i] = two_rings_cost(n, c1=modules1_per_ring, c2=modules2_per_ring)
    _, _, J_ind_even_list[i], _, _ = two_rings_cost(n+1, c1=modules1_per_ring, c2=modules2_per_ring)
    
    # compute actual objectives achieved by altmap
    G = generate_two_rings(n)
    G = nx.convert_node_labels_to_integers(G, first_label=1)
    communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True)
    J_altmap_list[i] = altmap_cost(G, communities_found)


In [24]:
fig, ax = plt.subplots(figsize=(12,9))
# plt.suptitle('Network of two rings - objective over ring size')
ax.plot(n_list, -J_true_list, 'x--', label='Ground truth')
ax.plot(n_list, -J_init_list, '^--', label='Each node as a module')
ax.plot(n_list, -J_ind_even_list, 'o--', label='Maximum independent sets - $n_c$ even')
ax.plot(n_list, -J_ind_odd_list, 'o--', label='Maximum independent sets - $n_c$ odd')
ax.plot(n_list, -J_c1_comms_list, 's--', label=f'{modules1_per_ring} modules per ring')
ax.plot(n_list, -J_c2_comms_list, 's--', label=f'{modules2_per_ring} modules per ring')
ax.plot(n_list, -J_altmap_list, '+--', ms=16, label='Implementation')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$')
ax.set_ylabel('Synthesizing Infomap objective $\mathcal{J}(\mathcal{Y}_c)$')
ax.legend()
ax.semilogx()
plt.tight_layout()

In [25]:
fig_path = '/home/chri/tworings_objective.pdf'
plt.savefig(fig_path, dpi=600, format='pdf')


In [22]:
nc = 50
results = []
for nc in [10,20, 30, 50, 70, 100]:
    J_c_comms_list, c_list = two_rings_cost_c_modules(nc)
    results.append({'nc':nc, 'J_list':J_c_comms_list, 'c_list':c_list})
        
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))

result = results[0]; nc=result['nc']
ax.plot(result['c_list'], result['J_list'], 'x--', label=f'$n_c$ = {nc}')
result = results[1]; nc=result['nc']
ax.plot(result['c_list'], result['J_list'], '^--', label=f'$n_c$ = {nc}')
result = results[2]; nc=result['nc']
ax.plot(result['c_list'], result['J_list'], 'o--', label=f'$n_c$ = {nc}')
result = results[3]; nc=result['nc']
ax.plot(result['c_list'], result['J_list'], 's--', label=f'$n_c$ = {nc}')
result = results[4]; nc=result['nc']
ax.plot(result['c_list'], result['J_list'], 'x--', label=f'$n_c$ = {nc}')
result = results[5]; nc=result['nc']
ax.plot(result['c_list'], result['J_list'], '^--', label=f'$n_c$ = {nc}')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Modules per ring $c$')
ax.set_ylabel('Synthesizing Infomap objective $\mathcal{J}(\mathcal{Y}_c)$')
ax.legend(loc=4)
plt.tight_layout()


In [23]:
fig_path = '/home/chri/tworings_mc.pdf'
plt.savefig(fig_path, dpi=600, format='pdf')

In [54]:
# generate network
n_ring = 5
N = 2*n_ring # num nodes
G = generate_two_rings(n_ring)

# run community detection
# communities_found, num_communities_found,_,_ = infomap(G, altmap=False)
communities_found, num_communities_found,_,_ = infomap(G, altmap=True)
# communities_found, num_communities_found,_,_ = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

cost = altmap_cost(G, communities_found)
print (f'Achieved cost L = {cost}')


OrderedDict([(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2)])
We found 2 communities.
Achieved cost L = -0.5658680991051533


In [55]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)

In [6]:

# generate network
n_ring = 10
N = 2*n_ring # num nodes
G = generate_two_rings(n_ring)
nodes_ids = list(range(1, N+1))
ring_half1_size = int(n_ring/2)
ring_half2_size = n_ring - ring_half1_size

n_ring_odd = n_ring+1
G_odd = generate_two_rings(n_ring_odd)
nodes_ids_odd = list(range(1, 2*n_ring_odd+1))

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))

# ground truth
labels = [1] * n_ring + [2] * n_ring
communities_true = dict(zip(nodes_ids, labels))

# independent sets
labels = [1,2] * n_ring
communities_ind_even = dict(zip(nodes_ids, labels))

# labels = [1,2] * int(n_ring/2) + [3] + [1,2] * int(n_ring/2) + [3]
labels = [1,2] * (int(n_ring_odd/2) - 1) + [1,3,2] + [1,2] * int(n_ring_odd/2) + [3]
communities_ind_odd = dict(zip(nodes_ids_odd, labels))

# c modules per ring
c=4
nm = int(n_ring/c); delta = n_ring-c*nm
labels = [module for module in range(1,delta+1) for i in range(0,nm+1)]
labels.extend([module for module in range(delta+1,c+1) for i in range(0,nm)])
labels.extend([module for module in range(c+1,2*c-delta+1) for i in range(0,nm)])
labels.extend([module for module in range(2*c-delta+1,2*c+1) for i in range(0,nm+1)])
communities_c_modules = dict(zip(nodes_ids, labels))


plt.close('all')

plt.figure(figsize=(5,8))
drawNetwork(G, communities_true)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_init)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_ind_even)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G_odd, communities_ind_odd)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_c_modules)
plt.tight_layout()

# fig, axs = plt.subplots(1,5)
# fig.suptitle(f'Sample two rings networks')
# 
# drawNetwork(G, communities_true, ax=axs[0])
# axs[0].set_xlabel('Ground truth')
# drawNetwork(G, communities_init, ax=axs[1])
# axs[1].set_xlabel('Initial partition')
# drawNetwork(G, communities_ind_even, ax=axs[2])
# axs[2].set_xlabel('Max independent sets\n$n_c$ even')
# drawNetwork(G_odd, communities_ind_odd, ax=axs[3])
# axs[3].set_xlabel('Max independent sets\n$n_c$ odd')
# drawNetwork(G, communities_c_modules, ax=axs[4])
# axs[4].set_xlabel(f'{c} modules per ring')